In [ ]:
from pydub import AudioSegment

path = "/data/codes/apa/water.mp3"
audio = AudioSegment.from_mp3(path)

In [ ]:
audio = audio.set_frame_rate(16000)

audio.export("/data/codes/apa/water.wav", format="wav")

In [ ]:
%cd /data/codes/apa/serve/
from src.utils.arpa import arpa_to_ipa
from multiprocessing.pool import Pool
import numpy as np
import requests
import librosa
import json
import ray

In [ ]:
wav_path = "/data/codes/apa/wav/She's paying for some jewelry at a checkout.wav"
transcript = "She's paying for some jewelry at a check out".upper()

waveform, sr = librosa.load(wav_path, sr=16000)

params = {
    "audio": waveform.tolist(),
    "transcript": transcript
}

In [ ]:
inputs = [params.copy() for i in range(1)]

@ray.remote
def send_query(params):
    response = requests.post("http://14.162.145.55:9999/scoring", json=params)
    return response

outputs = ray.get([send_query.remote(params) for params in inputs])
outputs = [json.loads(output.content) for output in outputs]


In [ ]:
import pandas as pd
import json
from glob import glob
from tqdm import tqdm

In [ ]:
path = "/data/codes/apa/dictionary"
words = glob(f'{path}/*.json')

In [ ]:
word2ipa = {}

for path in tqdm(words):
    with open(path, "r", encoding="utf-8") as f:
        content = json.load(f)
        for meaning_cases in content["meaning_cases"]:
            for word_type_case in meaning_cases["word_type_cases"]:
                word = content["word"]
                ipa = word_type_case["head_content"]["ipa"]

                if word not in word2ipa:
                    word2ipa[word] = {
                        "us": ipa["us"],
                        "uk": ipa["uk"]
                    }
                else:
                    word2ipa[word]["us"] += ipa["us"]
                    word2ipa[word]["uk"] += ipa["uk"]

In [ ]:
with open("cambridge-dict.json", 'w', encoding="utf-8")as f:
    json_obj = json.dumps(word2ipa, indent=4, ensure_ascii=False)
    f.write(json_obj)

In [ ]:
import librosa
import soundfile as sf
import os

In [ ]:
in_path = "/data2/data/asr_dataset/LibriSpeech/dev-clean/251/118436/251-118436-0021.flac"
out_path = os.path.basename(in_path).replace(".flac", ".wav")

In [ ]:
wav, sr = librosa.load(in_path, sr=16000)

sf.write(out_path, data=wav, samplerate=sr)